## Import packages and specify local GPU for training

In [5]:
import torch
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

from mpl_toolkits.axes_grid1 import make_axes_locatable
from torch.utils.data import Dataset, DataLoader, random_split
import torch.optim as optim

import EWOthello.utils.plot_helpers as plt_util
from EWOthello.data.othello import *
from EWOthello.mingpt.dataset import ProbingDataset, CharDataset # AK's mingpt data child 
from EWOthello.mingpt.model import GPT, GPTConfig, GPTforProbing, GPTforProbing_v2
from EWOthello.mingpt.probe_trainer import Trainer, TrainerConfig
from EWOthello.mingpt.utils import set_seed, sample # AKs helpers for sampling predictions
from EWOthello.mingpt.probe_model import BatteryProbeClassification
set_seed(44)

print(torch.cuda.is_available())
# device = "mps"
device = torch.cuda.current_device()
print(torch.cuda.get_device_name(device))

False


## Test Generation of Probe Datasets

In [6]:
othello = get(ood_num=-1, data_root=None, num_preload=1) # 11 corresponds to over 1 million games
game_dataset = CharDataset(othello) 

Max num files: 230; Use_num: 1
['gen10e5__20220324_155234.pickle']


Mem Used: 0.2633 GB: 100%|██████████| 1/1 [00:00<00:00,  1.79it/s]


Deduplicating...
Deduplicating finished with 100000 games left
Using 20 million for training, 0 for validation
Dataset created has 100000 sequences, 61 unique words.


In [7]:
mconf = GPTConfig(game_dataset.vocab_size, game_dataset.block_size, n_layer=8, n_head=8, n_embd=512)
model_probe = GPTforProbing(mconf, probe_layer=6)
model_probe.load_state_dict(torch.load("../EWOthello/ckpts/DeanKLi_GPT_Synthetic_8L8H/GPT_Synthetic_8Layers_8Heads.ckpt"))   
model_probe = model_probe.to(device)

properties_modifier_matrix = np.ones((59, 64))
for i in range(59):
    if i % 2 == 1:
        properties_modifier_matrix[i, :] *= -1.0

property_container_v2 = []
property_container = []
act_container = []
for x, _ in tqdm(game_dataset):
    # Convert the game index sequence to board number sequence for use with the othello board class
    tbf = [game_dataset.itos[_] for _ in x.tolist()]
    valid_until = tbf.index(-100) if -100 in tbf else 999

    # Get the board state vectors
    a = OthelloBoardState()
    properties = a.get_gt(tbf[:valid_until], "get_state")
    property_container.extend(properties)
    properties_v2 = (np.array(properties) - 1.0) * properties_modifier_matrix[:valid_until, :] + 1.0
    property_container_v2.extend(properties_v2.tolist())

    # Get the activation vectors
    act = model_probe(x[None, :].to(device))[0].detach().cpu().split(1, dim=0)    
    act_container.extend(act)
    break

### Visualize
num_disp = 5
fig = plt.figure(figsize=(num_disp*2, 4))
ax = plt_util.addAxis(fig, 2, num_disp)
for i in range(num_disp):
    plt_util.plot_game_discs(np.reshape(property_container[i],[8,8]), ax[i])
    plt_util.plot_game_discs(np.reshape(property_container_v2[i], [8,8]), ax[i+num_disp])
    ax[i].set_title(f"Moves Played: {i+1}")

plt.tight_layout()
ax[0].set_ylabel("Board State (Old)")
ax[num_disp].set_ylabel("New Representation")
plt_util.format_ax_boardImage(ax)

RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

## Train probes for all GPT models

In [7]:
othello = get(ood_num=-1, data_root=None, num_preload=11) # 11 corresponds to over 1 million games
game_dataset = CharDataset(othello)

class probe_dataset(Dataset):
    def __init__(self, mconf, gpt_ckpt_path, game_dataset, probe_layer, property_type="new"):
        self.game_dataset = game_dataset
        self.property_type = property_type # 'old' vs 'new'
        self.property_modifier = np.concatenate([np.ones((1,64))*(-1)**i for i in range(59)],axis=0)

        # Define the GPT probe model to return activations
        GPT_probe = GPTforProbing(mconf, probe_layer=probe_layer)
        GPT_probe.load_state_dict(torch.load(gpt_ckpt_path))
        print("Loaded GPT Model from: ", gpt_ckpt_path)
        for param in GPT_probe.parameters():
            param.requires_grad = False
        GPT_probe.eval()
        self.GPT_probe = GPT_probe.to(device)
    
    def __len__(self):
        return len(self.game_dataset)

    def __getitem__(self, index):
        x, _ = self.game_dataset[index]
        tbf = [game_dataset.itos[_] for _ in x.tolist()]
        valid_until = tbf.index(-100) if -100 in tbf else 999

        # Get the board state vectors
        a = OthelloBoardState()
        board_state = a.get_gt(tbf[:valid_until], "get_state")
        if self.property_type=="new":
            board_state = (np.array(board_state) - 1.0) * self.property_modifier[:valid_until, :] + 1.0

        # Get the activation vectors
        act = self.GPT_probe(x[None, :].to(device))[0, :valid_until, :].detach().cpu()
        
        return act, torch.tensor(board_state, dtype=torch.float32)

def train_linear_probe(training_data, model_name, probe_layer, savepath, property_type="New", num_epochs=1, train_to=1000, save_at_steps=100, verbose=False):
    probe = BatteryProbeClassification(device, probe_class=3, num_task=64, input_dim=512)
    train_ratio = 0.8
    batch_size = 512 # Batch over number of games (each game has 59 max moves played)
    learning_rate = 0.005

    def my_collate_fn(batch):
        xstack = []
        ystack = []
        for batchit in batch:
            xstack.append(batchit[0])
            ystack.append(batchit[1])
        
        x =  torch.cat(xstack, dim=0)
        y = torch.cat(ystack, dim=0)
        return (x,y)

    train_size = int(train_ratio * len(training_data))
    test_size = len(training_data) - train_size
    train_dataset, test_dataset = random_split(training_data, [train_size, test_size])
    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=my_collate_fn)
    test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, collate_fn=my_collate_fn)
    test_iter = iter(test_dataloader)
    optimizer = optim.Adam(probe.parameters(), lr=learning_rate)
    training_loss_history = []
    testing_loss_history = []
    testing_board_accuracy = []

    print(f"Training Model Name: {model_name}; training/test set size {train_size}/{test_size}")
    if os.path.exists(savepath + model_name + ".ckpt"):
        probe.load_state_dict(torch.load(savepath + model_name + ".ckpt"))
        print(f"Loaded model checkpopint from {savepath + model_name + '.ckpt'}")
        with open(savepath + model_name + ".pickle", 'rb') as fhandle:
            training_history = pickle.load(fhandle)
            training_loss_history = training_history["training_loss"]
            testing_loss_history = training_history["testing_loss"]
            testing_board_accuracy = training_history["testing_board_acc"]
    
    i = 0 if len(training_loss_history)==0 else len(training_loss_history)
    for epoch in range(num_epochs):
        for (x,y) in tqdm(train_dataloader):
            if i > train_to: # Resume at an interrupted step
                break

            ### Train the Probe
            probe.train()
            x = x.to(device)
            y = y.to(device)

            logits, loss = probe(x, y)
            train_loss = loss.item()
            training_loss_history.append(train_loss)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            ### Compute the error on validation batch
            probe.eval()
            with torch.no_grad():
                try:
                    x,y = next(test_iter)
                except:
                    test_iter = iter(test_dataloader)
                    x,y = next(test_iter)

                x = x.to(device)
                y = y.to(device)        
                logits, loss = probe(x,y)
                test_loss = loss.item()
                testing_loss_history.append(test_loss)

                y_hat = torch.argmax(logits, dim=-1, keepdim=False)  # [B, #task]
                mean_board_acc = torch.mean(torch.sum(y_hat == y ,-1)/64).item()
                testing_board_accuracy.append(mean_board_acc)

            ### Print update
            i = i + 1
            if verbose:
                print(f'step [{i+1}/{len(train_dataloader)}], Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}, Avg. Board Acc. Test {mean_board_acc:.3f}')

            if (i+1) % save_at_steps == 0:
                torch.save(probe.state_dict(), savepath + model_name + ".ckpt")
                training_history = {"training_loss": training_loss_history, "testing_loss": testing_loss_history, "testing_board_acc": testing_board_accuracy}
                with open(savepath + model_name + ".pickle", 'wb') as fhandle:
                    pickle.dump(training_history, fhandle)

    return

Max num files: 230; Use_num: 11
['gen10e5__20220324_165952.pickle', 'gen10e5__20220324_154919.pickle', 'gen10e5__20220324_164123.pickle', 'gen10e5__20220324_154043.pickle', 'gen10e5__20220324_155251.pickle', 'gen10e5__20220324_160016.pickle', 'gen10e5__20220324_165748.pickle', 'gen10e5__20220324_154002.pickle', 'gen10e5__20220324_155241.pickle', 'gen10e5__20220324_165707.pickle', 'gen10e5__20220324_160046.pickle']


Mem Used: 4.045 GB: 100%|██████████| 11/11 [00:05<00:00,  1.90it/s]


Deduplicating...
Deduplicating finished with 1099941 games left
Using 20 million for training, 0 for validation
Dataset created has 1099941 sequences, 61 unique words.


In [8]:
datapath = "../EWOthello/ckpts/"
all_files = os.listdir(datapath)
GPT_Folders = [name for name in all_files if name.startswith("Dean_GPTv2_Synthetic")]
GPT_Folders.sort()
print(GPT_Folders)

for fold in GPT_Folders:
    savepath = datapath + fold + "/"
    n_layer = int(fold[-4:-3])
    n_head = int(fold[-2:-1])
    print(n_layer, n_head, fold)
    
    gpt_ckpt_path = f'../EWOthello/ckpts/Dean_GPTv2_Synthetic_{n_layer}L{n_head}H/GPT_Synthetic_{n_layer}Layers_{n_head}Heads.ckpt'
    mconf = GPTConfig(game_dataset.vocab_size, game_dataset.block_size, n_layer=n_layer, n_head=n_head, n_embd=512)

    for probe_layer in np.arange(1, n_layer+1, 1):
        training_data = probe_dataset(mconf, gpt_ckpt_path, game_dataset, probe_layer)
        model_name = f"linearProbe_Map_New_{n_layer}L{n_head}H_GPT_Layer{probe_layer}"
        train_linear_probe(training_data, model_name, probe_layer, savepath, property_type="New", num_epochs=1, train_to=1000, save_at_steps=100, verbose=False)

['Dean_GPTv2_Synthetic_1L1H', 'Dean_GPTv2_Synthetic_1L8H', 'Dean_GPTv2_Synthetic_4L1H', 'Dean_GPTv2_Synthetic_4L8H', 'Dean_GPTv2_Synthetic_8L8H']
1 1 Dean_GPTv2_Synthetic_1L1H
Loaded GPT Model from:  ../EWOthello/ckpts/Dean_GPTv2_Synthetic_1L1H/GPT_Synthetic_1Layers_1Heads.ckpt
Training Model Name: linearProbe_Map_New_1L1H_GPT_Layer1; training/test set size 879952/219989


 58%|█████▊    | 1001/1719 [31:22<22:30,  1.88s/it]


1 8 Dean_GPTv2_Synthetic_1L8H
Loaded GPT Model from:  ../EWOthello/ckpts/Dean_GPTv2_Synthetic_1L8H/GPT_Synthetic_1Layers_8Heads.ckpt
Training Model Name: linearProbe_Map_New_1L8H_GPT_Layer1; training/test set size 879952/219989


 58%|█████▊    | 1001/1719 [29:47<21:22,  1.79s/it]


4 1 Dean_GPTv2_Synthetic_4L1H
Loaded GPT Model from:  ../EWOthello/ckpts/Dean_GPTv2_Synthetic_4L1H/GPT_Synthetic_4Layers_1Heads.ckpt
Training Model Name: linearProbe_Map_New_4L1H_GPT_Layer1; training/test set size 879952/219989


...

 58%|█████▊    | 1001/1719 [1:03:10<45:19,  3.79s/it]


Loaded GPT Model from:  ../EWOthello/ckpts/Dean_GPTv2_Synthetic_8L8H/GPT_Synthetic_8Layers_8Heads.ckpt
Training Model Name: linearProbe_Map_New_8L8H_GPT_Layer7; training/test set size 879952/219989


 58%|█████▊    | 1001/1719 [1:11:13<51:05,  4.27s/it] 


Loaded GPT Model from:  ../EWOthello/ckpts/Dean_GPTv2_Synthetic_8L8H/GPT_Synthetic_8Layers_8Heads.ckpt
Training Model Name: linearProbe_Map_New_8L8H_GPT_Layer8; training/test set size 879952/219989


 58%|█████▊    | 1001/1719 [1:20:20<57:37,  4.82s/it]  
